# todo
* increase chunk size - 5k+
* increase games count
* adaptive learning rate
* batch normalization
* focus on mid-end game
* more comment tips


In [3]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
import tqdm # type: ignore
from dataset import ChessDataset
from model import ChessModel
from helper_funcs import collect_unique_moves, create_input_for_nn
from helper_funcs import process_data_and_save_chunks

# Data Processing

## load data - into chunks so that memory is not overwhelmed, store them in sepearte folder

In [2]:
files = [os.path.join("../data/pgn", file) for file in os.listdir("../data/pgn") if file.endswith(".pgn")]
files.sort()  # Ensure consistent order
LIMIT_OF_FILES = min(len(files), 28)
files = files[:LIMIT_OF_FILES]

# Collect unique moves
move_to_int, num_classes = collect_unique_moves(files, max_games=50000)

In [4]:
data_chunk_files = process_data_and_save_chunks(files, move_to_int, chunk_size=5000, max_games=50000)

Processing Data Chunks:   0%|          | 0/1 [34:15<?, ?it/s]


## setup

In [5]:
# Check for GPU
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS backend on Apple Silicon (M2).")
else:
    device = torch.device('cpu')
    print("MPS backend not available. Using CPU.")

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using MPS backend on Apple Silicon (M2).


# Train

In [6]:
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    total_batches = 0
    for data_chunk_file in tqdm.tqdm(data_chunk_files, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # Load data chunk
        data = np.load(data_chunk_file)
        X = torch.tensor(data['X'], dtype=torch.float32)
        y = torch.tensor(data['y'], dtype=torch.long)
        # Create Dataset and DataLoader
        dataset = ChessDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=64, num_workers=4, shuffle=True)
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)  # Raw logits
            # Compute loss
            loss = criterion(outputs, labels)
            loss.backward()
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()
            total_batches += 1
        # Free up memory
        del X, y, dataset, dataloader
        # Clear cache on mac m2
        # if device == torch.device('mps'):
        #     torch.mps.current_process.cache_clear()

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)
    avg_loss = running_loss / total_batches
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Time: {minutes}m{seconds}s')

Epoch 1/10:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# Save the model
torch.save(model.state_dict(), "../models/mark1-10e.pth")